In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

'''
각 레이어에서 '훈련 피처 맵'과 '정분류 테스트 피처 맵'과 '오분류 테스트 피처 맵'은 모두 같은 형태이어야 한다.
예를 들어, 첫번째 레이어에서 훈련 피처 맵(100, 12, 12) 정분류 테스트 피처 맵(30, 12, 12), 오분류 테스트 피처 맵(20, 12, 12)
와 같이 인덱스 첫번째인 개수는 다르더라도 이차 넘파이 배열은 같아야 한다.
'''

class fmd():
    # 0: 훈련, 1: 정분류 테스트, 2: 오분류 테스트
    train=0; rtest=1; wtest=2
    data_types=[0, 1, 2]
    # 데이터 타입에 대한 이름
    data_name=["train_data", "rtest_data", "wtest_data"]
    # (이하) 대괄호로 쳐져있는 배열은 나중에 넘파이 배열이 될 수 있다.
    # K: 피처 맵의 개수, L: 레이어의 개수(0~L-1는 각 레이어의 인덱스)
    # N: 행의 크기    , M: 열의 크기
    K=[]; L=0; N=[]; M=[]
    # (이하) 훈련 데이터, 정분류 테스트 데이터, 오분류 테스트 데이터에서 데이터라는 용어를 생략하고 적었다.
    # 아래의 배열들은 모두 파이썬 배열이고 첫 번째 인자에 layer가 들어간다.
    # TFM_mean: 훈련 평균 피처 맵(베이스 피처 맵)
    # RFM_mean: 정분류 테스트 평균 피처 맵
    # WFM_mean: 오분류 테스트 평균 피처 맵
    TFM_mean=[]; RFM_mean=[]; WFM_mean=[]
    # TAM: 훈련 활성화 맵       , RAM: 정분류 테스트 활성화 맵
    # WAM: 오분류 테스트 활성화 맵
    TAM=[]; RAM=[]; WAM=[]
    # alpha는 거리 계산을 위한 인덱스를 고르기 위해 필요한 변수이다.
    # w_minus_r_max(= max(w-r))는 나중에 alpha 값에 의존하여 최대로 만들어진다.
    # alpha_min, alpha_max는 각 레이어에서의 alpha가 가질 수 있는 최소값 최대값을 나타낸 것이다.
    # alpha_slice[하이퍼 파라미터]은 alpha_min에서 alpha_max로 몇 번의의 간격으로 도착할지 알려주는 변수임.
    alpha=[]; alpha_slice=[]; w_minus_r_max=[]
    alpha_min=[]; alpha_max=[]
    # DAM_indexes는 나중에 거리 계산할 때 쓰이는 2차원 인덱스들의 집합이다.
    # 각 원소는 피처 맵의 한 원소의 인덱스를 나타낸다.
    # DAM: 거리 활성화 맵, DAM는 거리 계산을 위한 인덱스만 활성된 맵이다.
    # DAM_select[하이퍼 파라미터]는 DAM를 고르는 방법을 알려줌.
    DAM_indexes=[]; DAM=[]; DAM_select=[]
    # 오분류 테스트 각 인덱스에서의 거리에 대한 중간값, 평균, 표준편차, 최소값, 최대값에 대한 배열은
    # 각 middle, mean, std, min, max임
    # 인덱스마다 달라진는 값이니 총칭해서 'index_infos'라고 하자.
    middle=[]; mean=[]; std=[]; min=[]; max=[]
    # origin_data_dir: 데이터 증강하기 전의 원래 데이터와 관련된 디렉토리 경로
    # augmentation_data_dir: 데이터 증강 데이터와 관련된 디렉토리 경로
    origin_data_dir=""; augmentation_data_dir=""

    def __init__(self, origin_data_dir_):
        # 인덱스 간의 거리를 계산을 위한 Distance 람다 함수
        # 속성으로 바로 초기화하면 객체가 인자로 들어가서 의도하지 않은 에러가 남
        # 따라서, __init__에 정의함
        self.ED = lambda x, y: abs(x-y)
        # 데이터에 관한 디렉토리 경로
        # 훈련, 정분류 테스트, 오분류 테스트를 저장하는 디렉토리를 포함하는 디렉토리
        self.origin_data_dir = origin_data_dir_
        # 훈련을 저장하는 디렉토리
        self.train_dir=f"{self.origin_data_dir}/{self.data_name[self.train]}"
        # 정분류 테스트를 저장하는 디렉토리
        self.rtest_dir=f"{self.origin_data_dir}/{self.data_name[self.rtest]}"
        # 오분류 테스트를 저장하는 디렉토리
        self.wtest_dir=f"{self.origin_data_dir}/{self.data_name[self.wtest]}"

    def create_practice_data(self):
        '''
        아마도 나중에 adapter를 만들 때 사용될 수 있을 것 같음.
        '''
        # 연습 데이터 랜덤 시드 설정
        random_seed_id = 42
        np.random.seed(random_seed_id)
        # 연습 데이터 범위 설정
        random_start = 1; random_end = 1000000
        # K 각 데이터 종류마다의 개수
        K=np.array([1000, 200, 400])
        # L: 레이어의 개수, N: 레어어의 행의 개수, M: 레어어의 열의 개수
        L=2;
        layer_0 = np.array([12, 12]).reshape(1,2)
        layer_1 = np.array([36, 36]).reshape(1,2)
        layers = np.append(layer_0, layer_1, axis=0).T
        N = layers[0]
        M = layers[1]

        data_info = np.append(K, L)
        data_info = np.append(data_info, N)
        data_info = np.append(data_info, M)
        # 데이터 정보 저장, 이를 통해서 실제 데이터에 접근함
        os.system(f"mkdir {self.origin_data_dir}")
        np.save(f"{self.origin_data_dir}/data_info.npy", data_info)
        # 실제 데이터 저장
        for data_type in self.data_types:
            os.system(f"mkdir {self.origin_data_dir}/{self.data_name[data_type]}")
            for kth_data in range(K[data_type]):
                for lth_layer in range(L):
                    np.save(f"{self.origin_data_dir}/{self.data_name[data_type]}/{self.data_name[data_type]}_{kth_data}_{lth_layer}.npy",
                            np.random.randint(random_start, random_end, size=(N[lth_layer], M[lth_layer]))
                            - np.random.rand(N[lth_layer], M[lth_layer]))
                    
    def init_from_practice_data(self):
        self.set_data_info()
        self.set_FM_means()
        self.set_AMs()
        self.set_index_infos()

    def set_data_info(self):
        data_info = np.load(f"{self.origin_data_dir}/data_info.npy")

        self.K = data_info[:3] # K: 0~2
        self.L = data_info[3] # L: 3
        self.N = data_info[4:4+self.L] # N: 4~4+(L-1)
        self.M = data_info[4+self.L:] # M: 4+L ~ 4+L+(L-1)

    def set_FM_means(self):
        '''
        self.TFM_mean, self.RFM_mean, self.WFM_mean를 포인터와 같이 추적할 수 있다면
        일반화하여 가독성을 위해 코드를 3배 더 줄일 수 있을 것 같은데 어떻게 하는지 모르겠다.
        '''
        # 각 레이어의 피처 맵을 0으로 초기화하여 생성
        for l in range(self.L):
            TFM_mean_l = np.zeros((self.N[l],self.M[l]))
            self.TFM_mean.append(TFM_mean_l)
        # 파일에 저장된 훈련 데이터 피처 맵 가지고 와서 TFM_mean 만들기
        # 0번 째 데이터를 TFM_mean에 넣은 후
        for l in range(self.L): # 0의 데이터
            TFM_0_l = np.load(f"{self.train_dir}/{self.data_name[self.train]}_{0}_{l}.npy")
            self.TFM_mean[l] = self.TFM_mean[l] + TFM_0_l
        # 1~K번 째 데이터로 TFM_mean을 구한다.
        for k in range(1, self.K[self.train]): # 1 ~ K 까지의의 데이터
            for l in range(self.L):
                TFM_k_l = np.load(f"{self.train_dir}/{self.data_name[self.train]}_{k}_{l}.npy")
                self.TFM_mean[l] = (self.TFM_mean[l]*k + TFM_k_l)/(k+1)
        
        # 각 레이어의 피처 맵을 0으로 초기화하여 생성
        for l in range(self.L):
            RFM_mean_l = np.zeros((self.N[l],self.M[l]))
            self.RFM_mean.append(RFM_mean_l)
        # 파일에 저장된 훈련 데이터 피처 맵 가지고 와서 RFM_mean 만들기
        # 0번 째 데이터를 TFM_mean에 넣은 후
        for l in range(self.L): # 0의 데이터
            RFM_0_l = np.load(f"{self.rtest_dir}/{self.data_name[self.rtest]}_{0}_{l}.npy")
            self.RFM_mean[l] = self.RFM_mean[l] + RFM_0_l
        # 1~K번 째 데이터로 TFM_mean을 구한다.
        for k in range(1, self.K[self.rtest]): # 1 ~ K 까지의의 데이터
            for l in range(self.L):
                RFM_k_l = np.load(f"{self.rtest_dir}/{self.data_name[self.rtest]}_{k}_{l}.npy")
                self.RFM_mean[l] = (self.RFM_mean[l]*k + RFM_k_l)/(k+1)
        
        # 각 레이어의 피처 맵을 0으로 초기화하여 생성
        for l in range(self.L):
            WFM_mean_l = np.zeros((self.N[l],self.M[l]))
            self.WFM_mean.append(WFM_mean_l)
        # 파일에 저장된 훈련 데이터 피처 맵 가지고 와서 WFM_mean 만들기
        # 0번 째 데이터를 TFM_mean에 넣은 후
        for l in range(self.L): # 0의 데이터
            WFM_0_l = np.load(f"{self.wtest_dir}/{self.data_name[self.wtest]}_{0}_{l}.npy")
            self.WFM_mean[l] = self.WFM_mean[l] + WFM_0_l
        # 1~K번 째 데이터로 TFM_mean을 구한다.
        for k in range(1, self.K[self.wtest]): # 1 ~ K 까지의의 데이터
            for l in range(self.L):
                WFM_k_l = np.load(f"{self.wtest_dir}/{self.data_name[self.wtest]}_{k}_{l}.npy")
                self.WFM_mean[l] = (self.WFM_mean[l]*k + WFM_k_l)/(k+1)

    def set_AMs(self):
        # alpha, w_minus_r_max를 0으로 초기화
        self.alpha = np.zeros(self.L)
        self.w_minus_r_max = np.zeros(self.L)
        # alpha slice를 일단 1000으로 함
        for l in range(self.L):
            self.alpha_slice.append(1000)
        # alpha의 최소값과 최대값을 구함
        self.alpha_min = np.zeros(self.L)
        self.alpha_max = np.zeros(self.L)
        for l in range(self.L):
            self.alpha_min[l] = int(np.array([self.RFM_mean[l].min(),
                                         self.TFM_mean[l].min(),
                                         self.WFM_mean[l].min()]).min())
            self.alpha_max[l] = int(np.array([self.RFM_mean[l].max(),
                                         self.TFM_mean[l].max(),
                                         self.WFM_mean[l].max()]).max())
            
        # TAM, RAM, WAM을 0으로 초기화 함
        for l in range(self.L):
            TAM_l = np.zeros((self.N[l],self.M[l]))
            self.TAM.append(TAM_l)
        for l in range(self.L):
            RAM_l = np.zeros((self.N[l],self.M[l]))
            self.RAM.append(RAM_l)
        for l in range(self.L):
            WAM_l = np.zeros((self.N[l],self.M[l]))
            self.WAM.append(WAM_l)

        # w-r가 최대가 되는 alpha, TAM, RAM, WAM을 찾음
        for l in range(self.L):
            # range 부분 고칠 필요가 있음
            a_min_l = self.alpha_min[l]; a_max_l = self.alpha_max[l]
            a_slice_l = self.alpha_slice[l]
            interval_l = (a_min_l - a_max_l)/a_slice_l
            # range(a_slice_l+1) 해야 a_min_l 부터 a_max_l 까지 감
            for alpha_l in [a_min_l + interval*s for s in range(a_slice_l+1)]:
                # 각 AM에서 alpha보다 큰 인덱스 부분은 1, 작거나 같은 인덱스 부분은 0로 됨
                TAM_l = np.array(self.TFM_mean[l] > alpha_l, dtype="int32")
                RAM_l = np.array(self.RFM_mean[l] > alpha_l, dtype="int32")
                WAM_l = np.array(self.WFM_mean[l] > alpha_l, dtype="int32")

                TAM_l_minus_RAM_l = TAM_l - RAM_l
                TAM_l_minus_WAM_l = TAM_l - WAM_l

                r_l = len(np.where(TAM_l_minus_WAM_l == 1)[0])
                w_l = len(np.where(TAM_l_minus_WAM_l == 1)[0])
                
                if w_l - r_l > self.w_minus_r_max[l]:
                    # w-r가 이전의 w-r보다 클 때
                    # alpha, w-r, TAM, RAM, WAM를 최신화함.
                    self.w_minus_r_max[l] = w_l - r_l
                    self.alpha[l] = alpha_l
                    self.TAM[l] = TAM_l
                    self.RAM[l] = RAM_l
                    self.WAM[l] = WAM_l

        # TAM, RAM, WAM를 이용하여 DAM를 구함
        # DAM에 WAM를 deep copy 복사함
        for l in range(self.L):
            DAM_l = self.WAM[l].copy()
            self.DAM.append(DAM_l)
        # 일단 모든 레이어를 'and' 방식으로 함
        for l in range(self.L):
            self.DAM_select.append("and")
        # 선택하려는 방식('and', 'or', 등등)대로 DAM를 고름
        for l in range(self.L):
            if self.DAM_select[l] == "and":
                TAM_l_and_RAM_l = np.logical_and(self.TAM[l], self.RAM[l])
                np.place(self.DAM[l], TAM_l_and_RAM_l, 0)
            elif self.DAM_select[l] == "or":
                TAM_l_or_RAM_l = np.logical_or(self.TAM[l], self.RAM[l])
                np.place(self.DAM[l], TAM_l_or_RAM_l, 0)
        
        # DAM_indexes를 지정함
        for l in range(self.L):
            nonzero_DAM_l = np.nonzero(self.DAM[l])
            DAM_indexes_l = []

            for i in range(len(nonzero_DAM_l[0])):
                DAM_indexes_l.append((nonzero_DAM_l[0][i], nonzero_DAM_l[1][i]))

            self.DAM_indexes.append(DAM_indexes_l)
    
    def set_index_infos(self):
        pass

fmd1 = fmd(origin_data_dir_="./practice_data")
# fmd1.create_practice_data()
fmd1.init_from_practice_data()

In [12]:
for l in range(fmd1.L):
    print(fmd1.TFM_mean[l])

In [22]:
np.zeros(4)

array([0., 0., 0., 0.])

In [ ]:
import numpy as np

b = []

In [5]:
min = 3
max = 3252
s = 1000
inteval = (max - min)/s
for interval in [min + s*inteval for s in range(1000+1)]:
    print(interval)


3.0
6.2490000000000006
9.498000000000001
12.747
15.996
19.245
22.494
25.743000000000002
28.992
32.241
35.49
38.739000000000004
41.988
45.237
48.486000000000004
51.735
54.984
58.233000000000004
61.482
64.731
67.98
71.229
74.47800000000001
77.727
80.976
84.22500000000001
87.474
90.723
93.97200000000001
97.221
100.47
103.71900000000001
106.968
110.217
113.46600000000001
116.715
119.964
123.21300000000001
126.462
129.711
132.96
136.209
139.458
142.707
145.95600000000002
149.205
152.454
155.703
158.952
162.201
165.45000000000002
168.699
171.948
175.197
178.446
181.695
184.94400000000002
188.193
191.442
194.691
197.94
201.189
204.43800000000002
207.687
210.936
214.185
217.434
220.68300000000002
223.93200000000002
227.181
230.43
233.679
236.928
240.17700000000002
243.42600000000002
246.675
249.924
253.173
256.422
259.671
262.92
266.169
269.418
272.66700000000003
275.916
279.165
282.414
285.663
288.91200000000003
292.161
295.41
298.659
301.908
305.157
308.406
311.65500000000003
314.904
318.153

In [3]:
a = np.zeros((7,5))
len(a)

7